In [13]:
from twitch_listener import utils
from twitch_listener import listener

In [14]:
# Connect to Twitch
bot = listener.connect_twitch('twitch_pioneers', 
                             'oauth:zkekfhyhzhdl8ltxs6b8jdez7b2i6b', 
                             'capstone_pioneers')

chat_file_path = '/home/w210/Twitch-chat-pioneers/chat_listener'

# List of channels to connect to
# Channel names must be lowercase
channels_to_listen_to = ['xqc']

In [15]:
#returns list of live channels
utils.is_live(channels_to_listen_to)

['xqc']

In [16]:
# Scrape live chat data into raw log files. (Duration is seconds)
bot.listen(channels_to_listen_to, duration = 10) 

# Convert log files into .CSV format
#bot.parse_logs(timestamp = True, file_path = chat_file_path)

# Generate adjacency matrix
#bot.adj_matrix(weighted = False, matrix_name = "streamer_network.csv")

26e3d1bd-b10b-43d8-a489-0b8a163f2c56


In [17]:
import sqlite3
conn = sqlite3.connect('db.sqlite3')
test = conn.execute('''SELECT name FROM sqlite_schema 
                        WHERE type='table'
                        ORDER BY name;''')
test.fetchall()

[('asmongold',), ('chats_table',), ('debug',), ('hasanabi',), ('xqc',)]

In [18]:
#test = conn.execute('''drop table if exists hasanabi''')
test = conn.execute('''select * from chats_table order by date desc''')
test.fetchall()

[('2022-07-06 05:26:00.647067',
  'airsxo',
  'sleep noicely sodaSmile lilly',
  'xqc',
  '26e3d1bd-b10b-43d8-a489-0b8a163f2c56'),
 ('2022-07-06 05:26:00.647067',
  'airsxo',
  'BatChest OMG IS THIS RP?',
  'xqc',
  '26e3d1bd-b10b-43d8-a489-0b8a163f2c56'),
 ('2022-07-06 05:26:00.647067',
  'airsxo',
  'f',
  'xqc',
  '26e3d1bd-b10b-43d8-a489-0b8a163f2c56'),
 ('2022-07-06 05:26:00.647067',
  'airsxo',
  ':tf:',
  'xqc',
  '26e3d1bd-b10b-43d8-a489-0b8a163f2c56'),
 ('2022-07-06 05:26:00.647067',
  'airsxo',
  'F TO PAY RESPECTS',
  'xqc',
  '26e3d1bd-b10b-43d8-a489-0b8a163f2c56'),
 ('2022-07-06 05:26:00.629278',
  'airsxo',
  'sleep noicely sodaSmile lilly',
  'xqc',
  '6e373e78-1049-4aeb-9d31-af2cd4bf9767'),
 ('2022-07-06 05:26:00.629278',
  'airsxo',
  'BatChest OMG IS THIS RP?',
  'xqc',
  '6e373e78-1049-4aeb-9d31-af2cd4bf9767'),
 ('2022-07-06 05:26:00.629278',
  'airsxo',
  'f',
  'xqc',
  '6e373e78-1049-4aeb-9d31-af2cd4bf9767'),
 ('2022-07-06 05:26:00.629278',
  'airsxo',
  ':tf:',
 

In [6]:
from pathlib import Path
import re
import os
def parse_logs(channels = [], timestamp = True, remove_bots = False, file_path = ''):

        """
        Method for converting raw data from text logs into .CSV format.

        Parameters:
            timestamp (boolean, optional) 
                - Whether or not to include the timestamp of chat messages. 
                - Note: timestamps represent when message 
                    was retrieved, not sent
            channels (list, optional)     
                - List of channel usernames for whom the text logs 
                    will be parsed into csv format.
                - If none are specified, the channels that are 
                    currently joined will be parsed
            remove_bots (bool, optional)
                - Whether or not to exclude messages sent by common bot accounts
        """

        # Check if specific list of channels is given
#         if len(channels) == 0:
#             try:
#                 channels = self.joined
#             except:
#                 print("Please either connect to channels, \
#                       or specify a list of log files to parse.")
                
#         if Path(file_path).exists() is False:
#             try: 
#                 os.mkdir(file_path) 
#             except OSError as error: 
#                 print("Failed to make chats directory")  
        
        # Set up regex for hex decoding
        ESCAPE_SEQUENCE_RE = re.compile(r'''
            ( \\U........      # 8-digit hex escapes
            | \\u....          # 4-digit hex escapes
            | \\x..            # 2-digit hex escapes
            | \\[0-7]{1,3}     # Octal escapes
            | \\N\{[^}]+\}     # Unicode characters by name
            | \\[\\'"abfnrtv]  # Single-character escapes
            )''', re.UNICODE | re.VERBOSE)
        
        def decode_escapes(s):
            def decode_match(match):
                return codecs.decode(match.group(0), 'unicode-escape')
        
            return ESCAPE_SEQUENCE_RE.sub(decode_match, s)
        
        # Check if string given for channe;s
        if type(channels) == str:
            channels = [channels]
            
        # Retrieve data from logs
        for channel in channels:
            if not channel.endswith(".log"):
                filename = os.getcwd() + '/logs/' + channel + ".log"
            lines = []
            with open(filename) as f:
                for line in f:
                    if line not in lines:
                        lines.append(line)
                        
            # Separate the raw strings into separate messages 
            split_messages = []
            for line in lines:
                print(type(line))
                count = line.count('.tmi.twitch.tv PRIVMSG #')
                entryInfo = 'Your host is tmi.twitch.tv' in line or 'End of /NAMES list\\r\\n' in line
                if entryInfo:
                    pass
                
                elif count == 0:
                    pass
                elif count == 1 and not entryInfo:
                    if line.endswith('\\r\\n\'\n'):
                        split_messages.append(line[:-6])
                    else:
                        split_messages.append(line)     
                else:
                    for msg in _split_line(line):
                        split_messages.append(msg)
            # Parse username, message text and (optional) datetime
            data = []          
            for message in split_messages:
                username = None
                message_text = None
                datetime = None
                row = {}
                
                # Parse message text
                hash_channel_point = message.find("PRIVMSG #" + channel)
                slice_ = message[hash_channel_point:]
                
                slice_point = slice_.find(":") + 1
                message_text = slice_[slice_point:]
                decoded_txt = decode_escapes(message_text).encode('latin1').decode('utf-8')
                row['text'] = decoded_txt
                
                # Parse username
                b = message.find("b")
                exclam = message.find("!")
                username = message[b:exclam][3:]
                row['username'] = username
                
                # Parse timestamp 
                # (note: dates are in weirdo American format)
                if timestamp:
                    datetime = message[:23] 
                    row['timestamp'] = datetime
            
                # Store observations
                if remove_bots and row['username'] in self.botlist:
                    pass
                else:
                    data.append(row)
            
#             # Write data to file
#             if len(data) > 0:
#                 pd.DataFrame(data).to_csv(file_path + channel + ".csv", index = False)

def _split_line(line, firstLine = False):

    prefix = line[:28]        
    if firstLine:
        line = line.split('End of /NAMES list\\r\\n')[1]        
    splits = [message for ind, message in enumerate(line.split("\\r\\n")) 
              if 'PRIVMSG' in message or ind == 0] 
    for i, case in enumerate(splits):
        if firstLine or i != 0:
            splits[i] = prefix + splits[i]

    return splits

In [7]:
parse_logs(['hasanabi'], file_path = '/home/w210/Twitch-chat-pioneers/chat_listener')

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
